## Download Dataset

In [80]:
from datasets import load_dataset

dataset = load_dataset("qiaojin/PubMedQA", "pqa_labeled")

In [81]:
dataset

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 1000
    })
})

In [83]:
train_ds = dataset['train']
train_ds[0]

{'pubid': 21645374,
 'question': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?',
 'context': {'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.',
   'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), ce

In [99]:
df = train_ds.to_pandas()
df.head()

,pubid,question,context,long_answer,final_decision
0,21645374,Do mitochondria play a role in remodelling lac...,{'contexts': ['Programmed cell death (PCD) is ...,Results depicted mitochondrial dynamics in viv...,yes
1,16418930,Landolt C and snellen e acuity: differences in...,{'contexts': ['Assessment of visual acuity dep...,"Using the charts described, there was only a s...",no
2,9488747,"Syncope during bathing in infants, a pediatric...",{'contexts': ['Apparent life-threatening event...,"""Aquagenic maladies"" could be a pediatric form...",yes
3,17208539,Are the long-term results of the transanal pul...,{'contexts': ['The transanal endorectal pull-t...,Our long-term study showed significantly bette...,no
4,10808977,Can tailored interventions increase mammograph...,{'contexts': ['Telephone counseling and tailor...,The effects of the intervention were most pron...,yes


In [85]:
len(df)

1000

In [ ]:
# original samples to be used for training
original_df = df[500:1000]

## Generate Answer Perturbations

In [57]:
ANSWER_PROMPT = f"""\nQUESTION: \n{{question}}\n\nGOLD ANSWER:\n{{gold_answer}}\n\nEVIDENCE TEXT:\n{{evidence_text}}\n\nHow can we change the GOLD ANSWER subtly such that it would be wrong? The GOLD ANSWER should look like a valid answer for the QUESTION when the EVIDENCE TEXT would be unavailable. The perturbed answer should still give the impression of a valid answer, but inspection of the EVIDENCE_TEXT would reveal that the perturbed answer is factually wrong. Output the new answer and change made in JSON format with the key 'new answer' and 'change made'."""
print(ANSWER_PROMPT)


QUESTION: 
{question}

GOLD ANSWER:
{gold_answer}

EVIDENCE TEXT:
{evidence_text}

How can we change the GOLD ANSWER subtly such that it would be wrong? The GOLD ANSWER should look like a valid answer for the QUESTION when the EVIDENCE TEXT would be unavailable. The perturbed answer should still give the impression of a valid answer, but inspection of the EVIDENCE_TEXT would reveal that the perturbed answer is factually wrong. Output the new answer and change made in JSON format with the key 'new answer' and 'change made'.


In [58]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = ""

client = OpenAI()
model_name = "gpt-4-turbo"

def get_openai_response(prompt):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        response_format={ "type": "json_object" }
    )

    return response.choices[0].message.content

def apply_answer_perturber(context, question, answer):
    prompt = ANSWER_PROMPT.format(question=question, gold_answer=answer, evidence_text=context)
    response = get_openai_response(prompt)
    return response

In [55]:
# select the first 500 rows for generating answer perturbations
df = df[:500]

In [59]:
df.head()

,pubid,question,context,long_answer,final_decision
0,21645374,Do mitochondria play a role in remodelling lac...,{'contexts': ['Programmed cell death (PCD) is ...,Results depicted mitochondrial dynamics in viv...,yes
1,16418930,Landolt C and snellen e acuity: differences in...,{'contexts': ['Assessment of visual acuity dep...,"Using the charts described, there was only a s...",no
2,9488747,"Syncope during bathing in infants, a pediatric...",{'contexts': ['Apparent life-threatening event...,"""Aquagenic maladies"" could be a pediatric form...",yes
3,17208539,Are the long-term results of the transanal pul...,{'contexts': ['The transanal endorectal pull-t...,Our long-term study showed significantly bette...,no
4,10808977,Can tailored interventions increase mammograph...,{'contexts': ['Telephone counseling and tailor...,The effects of the intervention were most pron...,yes


In [60]:
import json
from tqdm import tqdm

responses, perturbed_answer, change_made = [], [], []

for idx, row in tqdm(df.iterrows()):
    question = row['question']
    answer = row['final_decision'].capitalize() + ". " + row['long_answer']
    context = row['context']['contexts']
    
    result = apply_answer_perturber(context, question, answer)
    responses.append(result)

    try:
        res = json.loads(result)
        perturbed_answer.append(res['new answer'])
        change_made.append(res['change made'])
        print(res['new answer'])
        print(res['change made'])
    except:
        perturbed_answer.append(None)
        change_made.append(None)
        print("Unable to parse!")

df['perturbed_answer'] = perturbed_answer
df['change_made'] = change_made

1it [00:09,  9.03s/it]

Yes. Results depicted mitochondrial dynamics in vivo as PCD progresses within the lace plant, and highlight the correlation of this organelle with other organelles during developmental PCD. To the best of our knowledge, this is the first report of mitochondria and chloroplasts moving on transvacuolar strands to form a linear structure along the cell walls during developmental PCD. Also, for the first time, we have shown the feasibility for the use of CsA in a whole plant system. Overall, our findings implicate the mitochondria as playing a critical and early role in developmentally regulated PCD in the lace plant.
Changed 'ring structure surrounding the nucleus' to 'linear structure along the cell walls'.


2it [00:15,  7.34s/it]

Yes. Using the charts described, there was a significant overestimation of visual acuity by the Snellen E compared to the Landolt C in strabismus amblyopia. Large differences in the lower visual acuity range have to be considered.
Changed 'No' to 'Yes' and 'only a slight overestimation' to 'a significant overestimation', and 'Small differences' to 'Large differences'. This alteration falsely suggests that the Snellen E chart greatly overestimates visual acuity compared to the Landolt C in strabismus amblyopia, contrary to the evidence provided.


3it [00:20,  6.51s/it]

...

Yes. Puberty and family factors were strong predictors of adolescent alcohol use, and family factors significantly moderated the association between pubertal stage and alcohol use.
Changed 'No' to 'Yes' and added that family factors significantly moderated the association, which is factually incorrect based on the evidence text.


499it [1:15:38,  4.65s/it]

Yes. Early oral intake is only possible after laparoscopy and not after laparotomy or colorectal resection, as the laparoscopic surgery uniquely allows for this benefit.
Changed 'No' to 'Yes' and added that early oral intake is only possible after laparoscopy, falsely claiming it as a unique benefit not applicable to laparotomy or colorectal resection.


500it [1:15:46,  9.09s/it]

Yes. The results of this study emphasize the importance of LOC as a predictor of neuropsychological test performance during the acute phase of recovery from mild traumatic brain injury. Neuropsychological testing procedures have been shown to be sensitive in measuring cognitive sequelae of mild traumatic brain injury (concussion) in athletes. The findings of this study indicate a significant relationship between LOC and neuropsychological functioning in a large sample of patients with mild head trauma, supporting the assignment of primary importance to LOC in grading severity of concussion. This study also supports the use of guidelines that rely heavily on LOC in making return-to-play decisions. Continued research is necessary to determine the relative importance of markers of concussion in athletes.
Changed the conclusion from 'No' to 'Yes' and altered the interpretation of the study's findings to incorrectly state that there was a significant relationship between LOC and neuropsycho

In [61]:
df.head()

,pubid,question,context,long_answer,final_decision,perturbed_answer,change_made
0,21645374,Do mitochondria play a role in remodelling lac...,{'contexts': ['Programmed cell death (PCD) is ...,Results depicted mitochondrial dynamics in viv...,yes,Yes. Results depicted mitochondrial dynamics i...,Changed 'ring structure surrounding the nucleu...
1,16418930,Landolt C and snellen e acuity: differences in...,{'contexts': ['Assessment of visual acuity dep...,"Using the charts described, there was only a s...",no,"Yes. Using the charts described, there was a s...",Changed 'No' to 'Yes' and 'only a slight overe...
2,9488747,"Syncope during bathing in infants, a pediatric...",{'contexts': ['Apparent life-threatening event...,"""Aquagenic maladies"" could be a pediatric form...",yes,Yes. 'Aquagenic maladies' are a pediatric form...,Added the phrase 'that always resolves complet...
3,17208539,Are the long-term results of the transanal pul...,{'contexts': ['The transanal endorectal pull-t...,Our long-term study showed significantly bette...,no,No. Our long-term study showed significantly b...,"In the original answer, it was stated that the..."
4,10808977,Can tailored interventions increase mammograph...,{'contexts': ['Telephone counseling and tailor...,The effects of the intervention were most pron...,yes,Yes. The effects of the intervention were most...,Switched the effectiveness of the intervention...


## Generate Reasoning chains

In [100]:
df = df[500:]
len(df)

500

In [101]:
from openai import OpenAI

client = OpenAI()

model_name = "gpt-4o"

def get_openai_response(SYSTEM_PROMPT, user_prompt):
    response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            response_format={ "type": "json_object" }
        )
    return response.choices[0].message.content

In [ ]:
SYSTEM_PROMPT = """You are given a QUESTION, CONTEXT, CHANGE MADE, GOLD ANSWER and ANSWER. Explain why the ANSWER is not faithful to the CONTEXT, given the QUESTION. CHANGE MADE specifies the change made to the GOLD ANSWER which made the ANSWER not faithful. Do not refer explicitly to the words 'CHANGE MADE' or 'GOLD ANSWER' in your reasoning. Generate your reasoning in JSON format: \n {\"REASONING\": \"<your reasoning steps as bullet points>\"}"""
user_prompt = """\n\n<QUESTION> \n{question}\n</QUESTION> \n\n<CONTEXT> \n{original_context}\n</CONTEXT> \n\n<CHANGE MADE> \n{change_made}\n</CHANGE MADE> \n\n <GOLD ANSWER> \n{answer}\n</GOLD ANSWER> \n\n <ANSWER> \n{new_answer}\n</ANSWER>"""

print(SYSTEM_PROMPT)
print(user_prompt)

In [103]:
reasonings, responses = [], []

In [ ]:
import json

for idx, row in tqdm(df.iterrows()):
    question = row['question']
    answer = row['final_decision'] + ". " + row['long_answer']
    context = row['context']['contexts']
    
    input_prompt = user_prompt.format(
        question=question,
        answer=answer,
        original_context=context,
        change_made=row['change_made'],
        new_answer=row['perturbed_answer']
    )
    response = get_openai_response(SYSTEM_PROMPT, input_prompt)
    responses.append(response)
    try:
        json_output = json.loads(response)
        reasoning = json_output['REASONING']
        reasonings.append(reasoning)
    except:
        print("Unable to parse!")
        reasonins.append(None)

In [ ]:
df['responses'] = responses
df['reasoning'] = reasonings
df.head()

In [110]:
df.to_json("../datasets/pubmedqa/perturbed_samples.json", lines=True, orient='records')

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, shuffle=True, test_size=0.2, random_state=42)
len(train_df), len(test_df)

In [ ]:
train_df, val_df = train_test_split(train_df, shuffle=True, test_size=0.25, random_state=42)
len(train_df), len(val_df)

In [34]:
train_df.drop(columns=['text'], inplace=True)
val_df.drop(columns=['text'], inplace=True)
test_df.drop(columns=['text'], inplace=True)

In [ ]:
train_df.head()

In [64]:
train_df['LABEL'] = 'PASS'
val_df['LABEL'] = 'PASS'
test_df['LABEL'] = 'PASS'


In [65]:
train_df.to_json("../datasets/drop/train_answer_perturbed_samples.jsonl", lines=True, orient='records')
val_df.to_json("../datasets/drop/val_answer_perturbed_samples.jsonl", lines=True, orient='records')
test_df.to_json("../datasets/drop/test_answer_perturbed_samples.jsonl", lines=True, orient='records')

## Reasoning chain for original samples

In [159]:
SYSTEM_PROMPT = """You are given a QUESTION, CONTEXT, ANSWER. Explain the similarities between the CONTEXT and the ANSWER. Reason about why the ANSWER is faithful to the CONTEXT given the QUESTION. Generate your reasoning in JSON format: \n {\"REASONING\": \"<your reasoning steps as bullet points>\"}"""
user_prompt = """\n\n<QUESTION> \n{question}\n</QUESTION> \n\n<CONTEXT> \n{original_context}\n</CONTEXT> \n\n <ANSWER> \n{answer}\n</ANSWER>"""

print(SYSTEM_PROMPT)
print(user_prompt)

You are given a QUESTION, CONTEXT, ANSWER. Explain the similarities between the CONTEXT and the ANSWER. Reason about why the ANSWER is faithful to the CONTEXT given the QUESTION. Generate your reasoning in JSON format: 
 {"REASONING": "<your reasoning steps as bullet points>"}


<QUESTION> 
{question}
</QUESTION> 

<CONTEXT> 
{original_context}
</CONTEXT> 

 <ANSWER> 
{answer}
</ANSWER>


In [161]:
from openai import OpenAI

client = OpenAI()

model_name = "gpt-4o"

def get_openai_response(SYSTEM_PROMPT, user_prompt):
    response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            response_format={ "type": "json_object" }
        )
    return response.choices[0].message.content

In [164]:
responses, reasonings = [], []

for idx, row in original_df.iterrows():
    gold_answer = row['final_decision'] + ". " + row['long_answer']
    input_prompt = user_prompt.format(
        question=row['question'],
        original_context=row['context']['contexts'],
        answer=gold_answer,
    )
    response = get_openai_response(SYSTEM_PROMPT, input_prompt)
    responses.append(response)
    reasonings.append(json.loads(response)['REASONING'])
    print(response)

{
  "REASONING": [
    "The CONTEXT discusses a study aimed at understanding the relationship between body mass index (BMI) and long-term survival in gastric cancer patients who have undergone gastrectomy.",
    "The study subgrouped patients into lower and higher BMI groups and compared their morbidity and long-term survival rates.",
    "The CONTEXT provides specific findings: a significantly longer mean survival rate for the lower BMI group in stage 2 and for the higher BMI group in stage 3a.",
    "The ANSWER states that BMI is a prognostic factor for stage 2 and stage 3a gastric cancer but not for other stages like 1a, 1b, 3b, and 4a.",
    "The ANSWER is faithful to the CONTEXT because it accurately reflects the study's findings that BMI influences survival rates in specific stages (2 and 3a) but does not generalize this influence to all stages of gastric cancer."
  ]
}
{
  "REASONING": [
    "The CONTEXT provides detailed information about the HINT1 gene, its location, and its p

In [ ]:
original_df['reasoning'] = reasonings
original_df['responses'] = responses

In [203]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(original_df, shuffle=True, test_size=0.2, random_state=42)
len(train_df), len(test_df)

(400, 100)

In [204]:
train_df, val_df = train_test_split(original_df, shuffle=True, test_size=0.25, random_state=42)
len(train_df), len(val_df)

(300, 100)

In [205]:
train_df.to_json("../datasets/pubmedqa/answer_perturbed_samples_train.jsonl", lines=True, orient='records')
val_df.to_json("../datasets/pubmedqa/answer_perturbed_samples_val.jsonl", lines=True, orient='records')
test_df.to_json("../datasets/pubmedqa/answer_perturbed_samples_test.jsonl", lines=True, orient='records')